# ETL com Python e Pandas - Transformando Dados de Contratos em Informações Valiosas

# Configuração do Ambiente e Importação de Bibliotecas

In [204]:
# !pip install -q pandas IPython
%pip install -q openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [205]:
import pandas as pd # A biblioteca pandas é a nossa principal ferramenta
import numpy as np
from IPython.display import Image # Image é usada para exibir imagens diretamente aqui no notebook.
import openpyxl as opl

In [179]:
def details_df(df, opc):
    try:
        if opc == 'tamanho':
            display(f'\nTamanho (linhas, colunas): {df.shape}')
        elif opc == 'linha':
            display(f'\nQuantidade Linhas: {df.shape[0]}')
        elif opc == 'coluna':
            nomes_colunas = df.columns.tolist()
            display(f'Quantidade de Colunas: {df.shape[1]}')
            display(f'Nomes das Colunas: {nomes_colunas}')
        elif opc == 'info':
            # Apenas chame df.info() diretamente
            print("\nInfo:")
            df.info()
        elif opc == 'head':
            # df.head() retorna um DataFrame, então imprima-o diretamente
            print("\nHead:")
            display(df.head())
        elif opc == 'isnull':
            # df.isnull().sum() retorna uma Series, então imprima-a diretamente
            print('\nIsnull:')
            display(df.isnull().sum())
        elif opc == 'todos':
            # Para 'todos', é melhor quebrar para não misturar print com retorno None
            print(f"\nTamanho (linhas, colunas):\n{df.shape}")
            print("\nInfo:")
            df.info()
            print("\nHead:")
            print(df.head())
            print("\nIsnull:")
            print(df.isnull().sum())
        elif opc == 'config':
            print('Informações sobre configuração detals_df:\nEstá função pode mostrar detalhes no campo opc:\ntamanho: Tamanho do DF.\nlinha: Quantidade de linhas no DF.' \
            '\ncoluna: Quantidade de colunas e nomes das colunas no DF.\ninfo: Puxa informações do DF.\nhead: Puxa o head (cabeçalho) de 5 linhas do DF.\nisnull: Informa campos vazios/nulos agrupado por colunas.' \
            '\ntodos: Puxa todas as informações. Obs: Menos linha e coluna.')
        else:
            print('Erro: Informe "opc" entre tamanho, linha, coluna, info, head, isnull ou todos')
    except Exception as e:
        print(f'Erro inesperado na função detals_df: {e}')
        

def describe_df(df,opc):
    try:
        if opc == 'normal':
            display(df.describe())
        elif opc == 'object':
            display(df.describe(include='object'))
        else:
            print('Erro: Informe campo "opc" entre normal e object!')
    except Exception as e:
            print(f'Erro: Inesperado na função describe_df: {e}')

def type_columns_df(df,opc):
    try:
        if opc == 'quantitativa':
            variaveis_quantitativas = df.select_dtypes(include=['number']).columns.tolist()
            print(f"Variáveis Quantitativas: {variaveis_quantitativas}")
            return variaveis_quantitativas
        elif opc == 'qualitativa':
            variaveis_qualitativas = df.select_dtypes(include=["category","object"]).columns.tolist()
            print(f"Variáveis Qualitativa: {variaveis_qualitativas}")
            return variaveis_qualitativas
        elif opc == 'todos':
            variaveis_quantitativas = df.select_dtypes(include=['number']).columns.tolist()
            print(f"Variáveis Quantitativas: {variaveis_quantitativas}")
            variaveis_qualitativas = df.select_dtypes(include=["category","object"]).columns.tolist()
            print(f"Variáveis Qualitativa: {variaveis_qualitativas}")
            # Retorna ambas as listas
            return variaveis_quantitativas, variaveis_qualitativas
        else:
            print('Erro: Escolha no campo "opc" entre "quantitativa", "qualitativa" ou "todos"')
            return None # Retorna None explicitamente para indicar falha ou opção inválida
    except Exception as e:
            print(f'Erro: Inesperado na função tipy_columns_df: {e}')
            return None 

def renames_columns_df(df,dicionario,boolean):
    try:
        if boolean:
            df.rename(columns=dicionario,inplace=boolean)
            print('Coluna(s) informada(s) no dicionário foram renomeadas com sucesso!')
            details_df(df,'head')
        elif boolean == False:
            df_final = df.rename(columns=dicionario)
            print('Coluna(s) informada(s) no dicionário foram renomeadas com sucesso!')
            details_df(df_final,'head')
            return df_final
        else:
            print('Erro: Informe o tipo de Inplace True ou False, caso True passe variável para receber informação.')
    except Exception as e:
        print(f'Erro: Inesperado na função renames_columns_df: {e}')
        return None 

def renames_fields_df(df,coluna,dicionario):
    try:
        df[coluna] =  df[coluna].replace(dicionario)
        print(df[coluna].value_counts(dropna=False))
        return df
    except Exception as e:
        print(f'Erro: Inesperado na função renames_fields_df: {e}')
        return None

def value_counts_df(df,opc,coluna):
    try:    
        if opc == 'normal':
            print(f'\n{df[coluna].value_counts(dropna=False)}')
        elif opc == 'proporcao':
            prop = df[coluna].value_counts(normalize=True).reset_index().rename(columns={"proportion":"Proporção"}).sort_values(by="Proporção", ascending=False)
            print(f'\n{prop}')
        elif opc == 'todos':
            print(f'\n{df[coluna].value_counts(dropna=False)}')
            print(f'\n{df[coluna].value_counts(normalize=True).reset_index().rename(columns={"proportion":"Proporção"}).sort_values(by="Proporção", ascending=False)}')
        elif opc == 'config':
            print('\nInformações sobre configuração value_counts_df:\nnormal: Retorna as categorias e suas quantidades de uma determina coluna.\nproporcao:  Retorna as categorias e suas proporção de uma determina coluna.\n' \
            'todos: Retorna as categorias, suas quantidades e proporção.')
        else:
            print('\nErro: Informe a "opc" entre normal, proporcao, todos e config para mais detalhes.')
    except Exception as e:
            print(f'Erro: Inesperado na função value_counts_df: {e}')

def str_upper_df(df,column):
    try:
        df[column] = df[column].str.upper()
        print(f'\nAs categorias contidas na coluna {column} foram convertidas para maiúsculas!')
    except Exception as e:
        print(f'Erro: Inesperado na função str_upper_df: {e}')

def drop_column_df(df,columns,boolean):
    try:
        if boolean:
            df.drop(columns=columns, inplace=boolean)
            print(f'Os campos {columns} foram excluídos com sucesso!')
            details_df(df,'head')
        elif boolean == False:
            df_final = df.drop(columns=columns, inplace=boolean)
            print(f'Os campos {columns} foram excluídos com sucesso!')
            details_df(df_final,'head')
            return df_final
        else:
            print('Erro: Informe o inplace True ou False, caso True passe variável para receber informação.')
    except Exception as e:
        print(f'Erro: Inesperado na função drop_column_df: {e}')


def convert_type_df(df,opc,coluna,tipo):
        try:
            if opc == 'outro':
                df[coluna] = df[coluna].astype(tipo)
                print(f'A coluna {coluna}, foi alterada para {tipo} com sucesso!')
                return df[coluna]
            elif opc == 'datetime':
                df[coluna]= pd.to_datetime(df[coluna], format='%d/%m/%Y')
                print(f'A coluna {coluna}, foi alterada para datetime com sucesso!')
                return df[coluna]
            else:
                print('Erro: Por favor escolha "opc" entre outros para definir tipos Ex: int ou datetime para conversão em datas.')
        except Exception as e:
            print(f'Erro: Inesperado na função convert_type_df: {e}')
            if opc == 'datetime':
                return df

# Extração de Dados (Extract)

A primeira etapa do ETL é a Extração. Vamos carregar cada um desses arquivos para um DataFrame do Pandas, que é a estrutura de dados principal que usaremos para manipular as tabelas.


In [3]:
path_tbcontratos = "./dados/tabela_contratos.csv"
path_tbdatas = "./dados/tabela_datas.csv"
path_tbempresas = "./dados/tabela_empresas.csv"

In [4]:
tbcontratos = pd.read_csv(path_tbcontratos)
tbdatas = pd.read_csv(path_tbdatas)
tbempresas = pd.read_csv(path_tbempresas)

In [11]:
details_df(tbcontratos,'head')
details_df(tbcontratos,'info')
details_df(tbcontratos,'isnull')


Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,fk_empresa_contratada
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,1
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,2
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,10,33,3
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,6,33,4
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,8,35,5



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_contrato            184 non-null    int64  
 1   nome_contrato          184 non-null    object 
 2   objeto_contrato        184 non-null    object 
 3   valor_contrato         184 non-null    float64
 4   inicio_vigencia        184 non-null    int64  
 5   termino_vigencia       184 non-null    int64  
 6   fk_empresa_contratada  184 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 10.2+ KB

Isnull:
id_contrato              0
nome_contrato            0
objeto_contrato          0
valor_contrato           0
inicio_vigencia          0
termino_vigencia         0
fk_empresa_contratada    0
dtype: int64


In [13]:
details_df(tbdatas,'head')
details_df(tbdatas,'info')
details_df(tbdatas,'isnull')


Head:


,id_data,data
0,1,20/05/2014
1,2,27/09/2014
2,3,28/11/2014
3,4,20/12/2014
4,5,28/12/2014



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id_data  254 non-null    int64 
 1   data     254 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.1+ KB

Isnull:
id_data    0
data       0
dtype: int64


In [43]:
#Realização do Merge com Join-Left pareando colunas fk_empresa_contratada e id_empresa, utilizando dataframes tbcontratos e tbempresas 
contratos_mod = tbcontratos.merge(tbempresas,
                                  left_on='fk_empresa_contratada',
                                  right_on='id_empresa',
                                  how='left')

In [44]:
details_df(contratos_mod,'head')
details_df(contratos_mod,'isnull')


Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,fk_empresa_contratada,id_empresa,nome_empresa
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,1,1,Thomas Jefferson
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,2,2,Chaveiro City
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,10,33,3,3,HBL Carimbos
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,6,33,4,4,Claro S/A
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,8,35,5,5,Imprensa Nacional



Isnull:


id_contrato              0
nome_contrato            0
objeto_contrato          0
valor_contrato           0
inicio_vigencia          0
termino_vigencia         0
fk_empresa_contratada    0
id_empresa               0
nome_empresa             0
dtype: int64

In [45]:
drop_column_df(contratos_mod,['fk_empresa_contratada','id_empresa'])

Os campos ['fk_empresa_contratada', 'id_empresa'] foram excluidos com sucesso!

Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,nome_empresa
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,Thomas Jefferson
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,Chaveiro City
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,10,33,HBL Carimbos
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,6,33,Claro S/A
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,8,35,Imprensa Nacional


## Combinando Contratos com Datas (Início de Vigência) - Exercício

In [51]:
contratos_final = contratos_mod.merge(tbdatas,
                                      left_on='inicio_vigencia',
                                      right_on='id_data',
                                      how='left')

In [52]:
details_df(contratos_final,'head')


Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,inicio_vigencia,termino_vigencia,nome_empresa,id_data,data
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,4,32,Thomas Jefferson,4,20/12/2014
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,13,33,Chaveiro City,13,26/02/2015
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,10,33,HBL Carimbos,10,10/02/2015
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,6,33,Claro S/A,6,01/01/2015
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,8,35,Imprensa Nacional,8,27/01/2015


In [53]:
drop_column_df(contratos_final,['inicio_vigencia','id_data'])

Os campos ['inicio_vigencia', 'id_data'] foram excluidos com sucesso!

Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,termino_vigencia,nome_empresa,data
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,32,Thomas Jefferson,20/12/2014
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,33,Chaveiro City,26/02/2015
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,33,HBL Carimbos,10/02/2015
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,33,Claro S/A,01/01/2015
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,35,Imprensa Nacional,27/01/2015


In [54]:
renames_columns_df(contratos_final,{'data':'inicio_vigencia'},True)

Colunas informadas no dicionário foram renomeadas com sucesso!

Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,termino_vigencia,nome_empresa,inicio_vigencia
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,32,Thomas Jefferson,20/12/2014
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,33,Chaveiro City,26/02/2015
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,33,HBL Carimbos,10/02/2015
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,33,Claro S/A,01/01/2015
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,35,Imprensa Nacional,27/01/2015


## Combinando Contratos com Datas (Término de Vigência)

In [180]:
# Combinamos o DataFrame 'contratos_final' (que já tem a data de início) novamente com o DataFrame 'datas' para obter as datas reais de término da vigência.
contratos_finalissima = contratos_final.merge(tbdatas,
                                              left_on='termino_vigencia',
                                              right_on='id_data',
                                              how='left')

In [181]:
details_df(contratos_finalissima,'head')


Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,termino_vigencia,nome_empresa,inicio_vigencia,id_data,data
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,32,Thomas Jefferson,20/12/2014,32,19/12/2015
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,33,Chaveiro City,26/02/2015,33,31/12/2015
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,33,HBL Carimbos,10/02/2015,33,31/12/2015
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,33,Claro S/A,01/01/2015,33,31/12/2015
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,35,Imprensa Nacional,27/01/2015,35,26/01/2016


In [182]:
drop_column_df(contratos_finalissima,['termino_vigencia','id_data'],True)

Os campos ['termino_vigencia', 'id_data'] foram excluídos com sucesso!

Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,inicio_vigencia,data
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,Thomas Jefferson,20/12/2014,19/12/2015
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,Chaveiro City,26/02/2015,31/12/2015
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,HBL Carimbos,10/02/2015,31/12/2015
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,Claro S/A,01/01/2015,31/12/2015
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,Imprensa Nacional,27/01/2015,26/01/2016


In [183]:
renames_columns_df(contratos_finalissima,{'data':'termino_vigencia'},True)

Coluna(s) informada(s) no dicionário foram renomeadas com sucesso!

Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,inicio_vigencia,termino_vigencia
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,Thomas Jefferson,20/12/2014,19/12/2015
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,Chaveiro City,26/02/2015,31/12/2015
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,HBL Carimbos,10/02/2015,31/12/2015
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,Claro S/A,01/01/2015,31/12/2015
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,Imprensa Nacional,27/01/2015,26/01/2016


In [184]:
print("Contagem de valores não nulos por coluna no DataFrame final:")
contratos_finalissima.count()

Contagem de valores não nulos por coluna no DataFrame final:


id_contrato         184
nome_contrato       184
objeto_contrato     184
valor_contrato      184
nome_empresa        184
inicio_vigencia     184
termino_vigencia    184
dtype: int64

In [185]:
print("Tipos de dados das colunas no DataFrame final antes da conversão de tipo de data:")
contratos_finalissima.dtypes

Tipos de dados das colunas no DataFrame final antes da conversão de tipo de data:


id_contrato           int64
nome_contrato        object
objeto_contrato      object
valor_contrato      float64
nome_empresa         object
inicio_vigencia      object
termino_vigencia     object
dtype: object

## Convertendo Colunas de Data para o Tipo Datetime

In [186]:
# Convertemos a coluna 'data_inicio_vigencia' para o formato de data.
# pd.to_datetime() é a função para isso.
# 'format='%d/%m/%Y'' especifica o formato da string de data (dia/mês/ano) para o Pandas interpretar corretamente.

contratos_finalissima['inicio_vigencia'] =  convert_type_df(contratos_finalissima,'datetime','inicio_vigencia','')


A coluna inicio_vigencia, foi alterada para datetime com sucesso!


In [187]:
details_df(contratos_finalissima,'info')
details_df(contratos_finalissima,'head')


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_contrato       184 non-null    int64         
 1   nome_contrato     184 non-null    object        
 2   objeto_contrato   184 non-null    object        
 3   valor_contrato    184 non-null    float64       
 4   nome_empresa      184 non-null    object        
 5   inicio_vigencia   184 non-null    datetime64[ns]
 6   termino_vigencia  184 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 10.2+ KB

Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,inicio_vigencia,termino_vigencia
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,Thomas Jefferson,2014-12-20,19/12/2015
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,Chaveiro City,2015-02-26,31/12/2015
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,HBL Carimbos,2015-02-10,31/12/2015
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,Claro S/A,2015-01-01,31/12/2015
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,Imprensa Nacional,2015-01-27,26/01/2016


In [188]:
contratos_finalissima['termino_vigencia'] = convert_type_df(contratos_finalissima,'datetime','termino_vigencia','')

Erro: Inesperado na função convert_type_df: day is out of range for month, at position 56. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


ValueError: Columns must be same length as key

In [189]:
# Veja qual é o valor exato que está causando o erro
print(contratos_finalissima['termino_vigencia'].iloc[56])

31/09/2017


In [190]:
contratos_finalissima.loc[contratos_finalissima['termino_vigencia']=='31/09/2017', 'termino_vigencia'] = '30/09/2017'

In [191]:
contratos_finalissima['termino_vigencia'] = pd.to_datetime(contratos_finalissima['termino_vigencia'], format='%d/%m/%Y')

In [192]:
details_df(contratos_finalissima,'info')
details_df(contratos_finalissima,'head')


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_contrato       184 non-null    int64         
 1   nome_contrato     184 non-null    object        
 2   objeto_contrato   184 non-null    object        
 3   valor_contrato    184 non-null    float64       
 4   nome_empresa      184 non-null    object        
 5   inicio_vigencia   184 non-null    datetime64[ns]
 6   termino_vigencia  184 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 10.2+ KB

Head:


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,inicio_vigencia,termino_vigencia
0,1,13/2012,Contratação de Empresa para Ministrar Curso de...,94947.23,Thomas Jefferson,2014-12-20,2015-12-19
1,2,Disp. Lic. 1,Prestação de serviço na confecção de Chaves,5234.00,Chaveiro City,2015-02-26,2015-12-31
2,3,Disp. Lic. 2,Prestação de serviço na confecção de Carimbos,1800.00,HBL Carimbos,2015-02-10,2015-12-31
3,4,32/2014,Serviço de Telefônia Móvel,73915.08,Claro S/A,2015-01-01,2015-12-31
4,5,001/2014,Publicações no Diário Oficial da União,45000.00,Imprensa Nacional,2015-01-27,2016-01-26


## Calculando o Tempo de Contrato  
Com as datas no formato correto, podemos agora criar uma métrica muito útil: o tempo_contrato. Essa nova coluna nos dirá a duração de cada contrato, em dias. Essa é uma transformação de "enriquecimento" de dados, onde criamos novas informações a partir das existentes.

In [193]:
contratos_finalissima['tempo_contrato'] = (contratos_finalissima['termino_vigencia'] -
                                           contratos_finalissima['inicio_vigencia']).dt.days

In [194]:
contratos_finalissima['tempo_contrato'] = convert_type_df(contratos_finalissima,'outro','tempo_contrato','int64')

# contratos_finalissima['tempo_contrato'].astype('int64')

A coluna tempo_contrato, foi alterada para int64 com sucesso!


In [195]:
details_df(contratos_finalissima,'info')


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_contrato       184 non-null    int64         
 1   nome_contrato     184 non-null    object        
 2   objeto_contrato   184 non-null    object        
 3   valor_contrato    184 non-null    float64       
 4   nome_empresa      184 non-null    object        
 5   inicio_vigencia   184 non-null    datetime64[ns]
 6   termino_vigencia  184 non-null    datetime64[ns]
 7   tempo_contrato    184 non-null    int64         
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 11.6+ KB


In [196]:
contratos_finalissima.describe()

,id_contrato,valor_contrato,inicio_vigencia,termino_vigencia,tempo_contrato
count,184.000000,1.840000e+02,184,184,184.000000
mean,92.500000,3.668124e+05,2017-11-23 15:46:57.391304448,2019-01-18 05:52:10.434782720,420.586957
min,1.000000,3.360000e+02,2014-05-20 00:00:00,2015-12-19 00:00:00,-2.000000
25%,46.750000,1.612710e+03,2016-02-10 00:00:00,2017-04-29 06:00:00,365.000000
50%,92.500000,3.391043e+04,2018-01-14 00:00:00,2019-03-01 00:00:00,365.000000
75%,138.250000,2.950292e+05,2019-10-05 00:00:00,2020-11-04 00:00:00,366.000000
max,184.000000,5.552089e+06,2020-09-08 00:00:00,2021-09-08 00:00:00,1096.000000
std,53.260367,8.721686e+05,NaN,NaN,194.561980


In [197]:
contratos_finalissima[contratos_finalissima['tempo_contrato']<1]

,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,inicio_vigencia,termino_vigencia,tempo_contrato
99,100,004/16,Serviços de Telefonia Móvel (Celulares),67461.0,Claro S/A,2019-05-01,2019-05-01,0
100,101,Disp. Lic. 34,Assinatura da Revista Época,418.8,Editora Globo S/A,2019-05-13,2019-05-11,-2
174,175,Disp. Inex. 60,Assinatura da Revista Época,454.8,Editora Globo S/A,2020-06-25,2020-06-25,0


In [198]:
# Filtramos o DataFrame para manter apenas os contratos onde 'tempo_contrato' é maior que 0 dias.
contratos_finalissima_filter = contratos_finalissima[contratos_finalissima['tempo_contrato']>0]

In [199]:
contratos_finalissima_filter.describe()

,id_contrato,valor_contrato,inicio_vigencia,termino_vigencia,tempo_contrato
count,181.000000,1.810000e+02,181,181,181.000000
mean,91.955801,3.725147e+05,2017-11-12 14:11:16.243094016,2019-01-14 03:50:43.093922560,427.569061
min,1.000000,3.360000e+02,2014-05-20 00:00:00,2015-12-19 00:00:00,179.000000
25%,46.000000,1.800000e+03,2016-01-29 00:00:00,2017-04-24 00:00:00,365.000000
50%,91.000000,3.495000e+04,2018-01-13 00:00:00,2019-01-31 00:00:00,365.000000
75%,138.000000,3.065407e+05,2019-10-05 00:00:00,2020-11-04 00:00:00,366.000000
max,184.000000,5.552089e+06,2020-09-08 00:00:00,2021-09-08 00:00:00,1096.000000
std,53.339669,8.782563e+05,NaN,NaN,188.357792


In [200]:
# Se quisermos inspecionar um 'nome_contrato' específico, por exemplo '004/16'.
# Isso pode revelar se há múltiplos contratos com o mesmo nome, valores diferentes, etc.

print("Detalhes para o 'nome_contrato' == '004/16':")
display(contratos_finalissima_filter[contratos_finalissima_filter.nome_contrato == '004/16'])

Detalhes para o 'nome_contrato' == '004/16':


,id_contrato,nome_contrato,objeto_contrato,valor_contrato,nome_empresa,inicio_vigencia,termino_vigencia,tempo_contrato
49,50,004/16,Serviços de Telefonia Móvel (Celulares),67461.0,Claro S/A,2016-05-01,2017-05-01,365
71,72,004/16,Serviços de Telefonia Móvel (Celulares),67461.0,Claro S/A,2017-05-01,2018-05-01,365
123,124,004/16,Serviços de Telefonia Móvel (Celulares),67461.0,Claro S/A,2019-05-01,2020-05-01,366
167,168,004/16,Serviços de Telefonia Móvel (Celulares),62780.6,Claro S/A,2020-05-01,2021-05-01,365


# Carregamento de Dados (Load)

**Chegamos à última fase do ETL**: o Carregamento. Aqui, pegamos nosso DataFrame contratos_finalissima, que está limpo, combinado e enriquecido, e o movemos para o seu destino final. Este destino pode ser um arquivo CSV para ser usado em Excel ou Google Sheets, um banco de dados local (como SQLite), ou um Data Warehouse em nuvem, como o Google BigQuery.

In [206]:
contratos_finalissima_filter.to_excel('etl_contrato.xlsx', index=False)
contratos_finalissima_filter.to_csv('etl_contrato.csv', index=False)